### CODE DOES NOT COMPILE DUE TO REMOVAL OF SOME INFORMATION FOR DISPLAY PURPOSE ONLY

In [ ]:
from pathlib import Path
import pandas as pd
import win32com.client as win32
import requests
import os
import codecs
from pretty_html_table import build_table
import base64
from datetime import date
import sys

In [ ]:
#This code grabs what day of the week it is
from datetime import datetime
waffle = datetime.today().isoweekday()

In [ ]:
EXCEL_FILE_PATH = r'C:\path./file name.xlsx'


#Get path to attachments and double check it's working
ATTACHMENT_DIR = Path.cwd() / "Attachments"
ATTACHMENT_DIR.mkdir(exist_ok=True)

In [ ]:
#Update file date here
data = pd.read_excel(EXCEL_FILE_PATH)
dtn = pd.read_csv('STARTING EXCEL FILE.csv', header=None) 
dtn.head(2)

In [ ]:
dir = 'S:\file path\Attachments'
for f in os.listdir(dir):
    os.remove(os.path.join(dir, f))

In [ ]:
if waffle != 1:
    data = data[data.Customer != 'monday cust']

In [ ]:
data = data[data.Customer != 'master list']

In [ ]:
dtn = dtn.drop([0,1,3,4,5,7,9,10,18,19,20,21], axis=1)
dtn.columns =['Company', 'Location', 'State','Prod', 'Product','Date','Time','Price1','Price2','Unknown','Unknown2']

In [ ]:
#Sort by oldest to newest
dtn = dtn.sort_values(by="Date", ascending=True)

In [ ]:
hold = dtn[dtn.Company != 'company to be exlcuded']
hold = dtn.loc[dtn['Company'] == 'random company']

In [ ]:
dtn['Date'] = dtn.apply(lambda x: x['Date']-1 if x['Company']=='Opis Average' else x['Date'], axis=1)

In [ ]:
# Gets prior day date from dtn file (top row, 5th column after drops)
ya = hold.iat[0,5]
dtn = dtn[dtn.Date == ya]

In [ ]:
# Create Marathon and BP dfs, insert default columns for later merge
place_one= dtn[dtn.Company != 'company to be exlcuded']
place_one = dtn.loc[dtn['Company'] == 'place one']
place_one.insert(11, "Default", 'place one', True)
oil = dtn.loc[dtn['Location'] == 'Oil Products']
oil.insert(11, "Default", 'Oil Products', True)
place_one.head(2)

In [ ]:
# Gets OPIS prices
opis = dtn.loc[dtn['Company'] == 'OPIS Average']
opis.insert(11, "Default", 'Opis', True)

In [ ]:
# best price of place one and oil
BPM = dtn.loc[(dtn['Location'] == 'Oil Products') | (dtn['Company'] == 'place one')]

In [ ]:
# Create 'Best of' df
BPM = BPM.sort_values(by="Price1", ascending=True)
BPM = BPM.drop_duplicates(subset=['State','Prod'])
BPM.insert(11, "Default", 'Best of', True)
BPM.head(2)

In [ ]:
#Best of by state, only for informational purposes
bestwa = dtn.loc[dtn['State'] == 'WA']
bestwa =bestwa.sort_values(by="Price1", ascending=True)
bestwa = bestwa.drop_duplicates(subset=['Prod'])
bestwa.insert(11, "Default", 'Bestwa', True)


bestor = dtn.loc[dtn['State'] == 'OR']
bestor =bestor.sort_values(by="Price1", ascending=True)
bestor = bestor.drop_duplicates(subset=['Prod'])
bestor.insert(11, "Default", 'Bestor', True)

In [ ]:
# Concats all dfs, prep for merge
pdList = [place_one, oil,bestwa,bestor,BPM,opis] 
yaaa = pd.concat(pdList)

In [ ]:
# Merge TNT Customer Pricing master list with concated df
data = data.merge(yaaa, how='left', on= ['Prod','State','Default'])
data = data.fillna(0)
data.head(2)

In [ ]:
#TCA and tax calculations (Unknown2=TCA discount)
data['Base Price'] = data['Price1'] - data['Unknown2'] + data['Margin']
data['State Tax'] = data['WA State Tax'] + data['WA State Env']
data['Sales Tax'] = (data['Base Price'] * data['State Sales Rate']) + (data['Base Price'] * data['PPT Rate'])
data['State Tax'] = data['Sales Tax'] + data['State Tax']
data['Fed Tax'] = data['Federal Excise'] +  data['LUST'] + data['Fed Oil Spill']

data = data.drop(['Price1', 'Unknown','Unknown2','Price2','Time','Date','Prod','Location','Company'], axis=1)
data = data.drop(['Email','Default','Margin','State','Federal Excise'], axis=1)
data = data.drop(['LUST','WA State Tax','Sales Tax','WA State Env','Fed Oil Spill','State Sales Rate','PPT Rate','Product_y'], axis=1)

data.rename(columns = {'Product_x':'Product Desc'}, inplace = True)
data['Total'] = data['Fed Tax'] + data['State Tax'] + data['Base Price']

data.head(2)

In [ ]:
# Details as str, remove blank details, combine prod & details
data["Details"] = " - "+ data["Details"].astype(str)
data.loc[(data.Details == ' - 0'),'Details']=''
data["Product Desc"] = data["Product Desc"].astype(str) + data["Details"].astype(str)
data.head(1)

In [ ]:
data = data.drop(['COMPANY Product','Details'], axis=1)

In [ ]:
data.rename(columns = {'Base Price':'Base Price'}, inplace = True)
data.rename(columns = {'Total':'Total Price'}, inplace = True)
data.head(1)

In [ ]:
# Create array of unique customer values
column_name = "Customer"
unique_values = data[column_name].unique()
unique_values

In [ ]:
for unique_value in unique_values:
    data_output = data.query(f"{column_name} == @unique_value")
    data_output = data_output.drop(['Customer'], axis=1)
    data_output['Base Price'] = data_output['Base Price'].astype('float')
    data_output['Base Price'] = ['$ {:,.3f}'.format(i) for i in list(data_output['Base Price'])]
    data_output['State Tax'] = data_output['State Tax'].astype('float')
    data_output['State Tax'] = ['$ {:,.3f}'.format(i) for i in list(data_output['State Tax'])]
    data_output['Fed Tax'] = data_output['Fed Tax'].astype('float')
    data_output['Fed Tax'] = ['$ {:,.3f}'.format(i) for i in list(data_output['Fed Tax'])]
    data_output['Total Price'] = data_output['Total Price'].astype('float')
    data_output['Total Price'] = ['$ {:,.3f}'.format(i) for i in list(data_output['Total Price'])]
    
    pretty = build_table(data_output,
                   'red_light',
                   width = '200',
                    text_align='center',
                    font_size = 'small',
                    padding = '2px')
    output_path = ATTACHMENT_DIR / f"{unique_value}_2022.html"
    with open(output_path, 'w') as f:
        f.write(pretty)

In [ ]:
# Same as previous step but for all customers to send internally
data['Base Price'] = data['Base Price'].astype('float')
data['Base Price'] = ['$ {:,.3f}'.format(i) for i in list(data['Base Price'])]
data['State Tax'] = data['State Tax'].astype('float')
data['State Tax'] = ['$ {:,.3f}'.format(i) for i in list(data['State Tax'])]
data['Fed Tax'] = data['Fed Tax'].astype('float')
data['Fed Tax'] = ['$ {:,.3f}'.format(i) for i in list(data['Fed Tax'])]
data['Total Price'] = data['Total Price'].astype('float')
data['Total Price'] = ['$ {:,.3f}'.format(i) for i in list(data['Total Price'])]
    
pretty = build_table(data,
                'red_light',
                 width = '200',
                 text_align='center',
                font_size = 'small',
                 padding = '2px')
output_path = ATTACHMENT_DIR / f"MASTERLIST_2022.html"
with open(output_path, 'w') as f:
     f.write(pretty)

## Slow down!

In [ ]:
display(data)
while True:

    number = input("Do you want to continue? Y or N ")

    if number == "Y":
        print("Looks good!!")
        break
    else:
        Exit

In [ ]:
#Creates email list, drops Monday only customers if not Monday
email_list = pd.read_excel(EXCEL_FILE_PATH)
email_list = email_list.drop_duplicates(subset=['Customer'])
email_list
if waffle != 1:
    email_list = email_list[email_list.Customer != 'monday cust']

In [ ]:
# Get today's date for email subject line
today = date.today()
d3 = today.strftime("%m/%d/%y")

In [ ]:
# Send emails
outlook = win32.Dispatch("outlook.application")
for index, row in email_list.iterrows():
    mail = outlook.CreateItem(0)
    mail.To = row["Email"]
    #mail.CC = row["CC"]
    mail.Subject = f"TITLE | " + d3
    # mail.Body = "Message body"
    signatureimage = "S:\DIRECTORY \LOGO.png"
    attachment = mail.Attachments.Add(signatureimage)
    attachment.PropertyAccessor.SetProperty("http://schemas.microsoft.com/mapi/proptag/0x3712001F", "MyId1")
    attachment_path = str(ATTACHMENT_DIR / f"{row['Customer']}_2022.html")
    with open (attachment_path, 'r') as file:
        data = file.read()               
    mail.HTMLBody ="<b>Hello " +row['Customer']+ data + "<html><img src=""cid:MyId1""></body></html>" 
                     
    mail.Display()
    
    mail.Send()